# Final Masters Project

## Name: Sreekanth Palagiri, Student ID: R00184198

## Project Topic: Evaluation of Ensemble Approach for Sentiment Analysis on a Small Dataset

##NoteBook1: Trainer Bert


### **Mount google drive**

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!ls "gdrive/My Drive/Colab Notebooks/Masters Project"

'Airline Tweets dataset'  'Sentence Polarity Dataset'
 glove.6B.300d.txt	   VMDataset


### **Install Flair**

In [3]:
!pip install transformers

     |████████████████████████████████| 2.1MB 7.7MB/s 
     |████████████████████████████████| 3.3MB 52.4MB/s 
     |████████████████████████████████| 901kB 51.6MB/s 


### **Setting Seed**

In [4]:
import random
import torch
import numpy as np

seed_val = 1               #the seed values ensures the same samples go in same order while training
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

### **Load Data and Preprocess**

In [5]:
import pandas as pd
import numpy as np

df=pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/Masters Project/VMDataset/Export_loop-sentiment-pos-neg-train_05112020000000.csv")
print(df.groupby(['label']).size())
df.head()

label
Negative     887
Positive    1013
dtype: int64


,label,text
0,Negative,No one cares about marketing slides - a techni...
1,Positive,Are all three hosts providing storage/capacity...
2,Negative,would loved to had managed to get down to the ...
3,Negative,Vending machine at work is out of Dasani water...
4,Positive,"RT @VMwareEdu: Paul Maritz, CEO and President ..."


**Preprocessor to Remove all special characters except emoticons**

In [6]:
import re

def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[^A-Za-z0-9\']+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

print(df['text'][19])
print(preprocessor(df['text'][19]))

Kristina,  Any updates from your side ? I volunteer for beta test :)  - really need that app running as workaround are driving me nuts ....
kristina any updates from your side i volunteer for beta test really need that app running as workaround are driving me nuts :)


In [7]:
df['text'] = df['text'].apply(preprocessor)

### **Label Encoding the Sentiment**

In [8]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
df['Sentiment']=le.fit_transform(df['label'])
df.head(10)

,label,text,Sentiment
0,Negative,no one cares about marketing slides a technica...,0
1,Positive,are all three hosts providing storage capacity...,1
2,Negative,would loved to had managed to get down to the ...,0
3,Negative,vending machine at work is out of dasani water...,0
4,Positive,rt vmwareedu paul maritz ceo and president of ...,1
5,Positive,had few folks ask if you're interested johnny ...,1
6,Positive,get notified of the latest vsan patch releases...,1
7,Negative,end of general support is 3 12 2020 6 5 and 6 ...,0
8,Negative,placed 4th in funrun today in the 17 39 age gr...,0
9,Positive,yup guys being currently under nda know this f...,1


In [9]:
possible_labels = df['Sentiment'].unique() 
len(possible_labels)

2

### **Split into Train and test datasets**

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test= train_test_split(df['text'], 
                                                   df['Sentiment'], 
                                                   random_state=1, 
                                                   test_size=0.15, 
                                                   shuffle=False)

### **Bert Model**

**Define Bert Tokenixer from pre-trained models**

In [11]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True) 

**Encode Test and Train DataSets**

In [12]:
encoded_data_train=tokenizer.batch_encode_plus(   
                        X_train.values,    
                        add_special_tokens=True,     
                        return_attention_mask=True,  
                        padding='longest', 
                        truncation=True,    
                        max_length=256,   
                        return_tensors='pt') 


encoded_data_test=tokenizer.batch_encode_plus(
                        X_test.values,              # Same we are doing for validation set.
                        add_special_tokens=True,
                        return_attention_mask=True,
                        padding='longest',
                        max_length=256,
                        truncation=True,
                        return_tensors='pt')

### **Created Torch Datasets in the format required for HuggingFace**

In [13]:
from torch.utils.data import TensorDataset

input_ids_train= encoded_data_train['input_ids']
input_ids_test= encoded_data_test['input_ids']


attention_masks_train= encoded_data_train['attention_mask']
attention_masks_test= encoded_data_test['attention_mask']


labels_train= torch.tensor(Y_train.values)
labels_test= torch.tensor(Y_test.values)


dataset_train= TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_test= TensorDataset(input_ids_test, attention_masks_test, labels_test)

print(len(dataset_train),len(dataset_test))


1615 285


**Load Predefined Bert Model for Classification**

In [14]:
from transformers import BertForSequenceClassification 

model= BertForSequenceClassification.from_pretrained(
        'bert-base-uncased',                               #BERT pre-trained model
        num_labels= len(possible_labels),
        output_attentions=False,
        output_hidden_states=False
    )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

**Prepare Torch DataLoader for Training Process**

In [15]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup

batch_size=4 
epochs= 8

dataloader_train= DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)
# Creating dataloader_train variable as dataloader object passing tokenized train data, sampling parameter and batch size.

dataloader_test= DataLoader(
    dataset_test,
    sampler=RandomSampler(dataset_test),
    batch_size= 32
)

optimizer = AdamW(
    model.parameters(), #it will take all parameter we defined in variable model above.
    lr=2.0e-5,      # acc to paper learning rate is 2e-5>5e-5 hyperparameter. We can vary this and check foe best accuracy.
    eps=1e-8        # eps stands for epsilon and this is default value.
)

scheduler= get_linear_schedule_with_warmup(
    optimizer,                                        # passing optimizer parameters
    num_warmup_steps=0,                               # default #lower learning rate
    num_training_steps=len(dataloader_train)*epochs   # As training steps is total training in all epochs therefore multiply

)

**Checking Availability for GPU**

In [16]:
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


**Define Evaluation Function**

In [17]:
from tqdm.notebook import tqdm  

def evaluate(dataloader_val):

    model.eval()                            # calling the builtin function eval present in BERT classification.
    
    loss_val_total = 0
    predictions, true_vals = [], []         # creating empty lists for appending the values.
    
    for batch in tqdm(dataloader_val):      # for all validation samples or rows.
        
        batch = tuple(b.to(device) for b in batch) # Passing the batch of samples to device as we initialised model.todevice
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],  #passing the input of dataloader_val
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()       # calculating the loss for each batch

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)            # appending fucntion to append all result in the list
        true_vals.append(label_ids)           # appending fucntion to append all result in the list
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0) # forming a prediction array
    true_vals = np.concatenate(true_vals, axis=0)     # forming true label array
            
    return loss_val_avg, predictions, true_vals

### **Training Bert Model**


In [18]:
from sklearn.metrics import accuracy_score

def scoring_func(preds, labels):
    preds_flat= np.argmax(preds, axis=1).flatten() # as prediction array is flattened to 0 or 1, possible labels
    labels_flat=labels.flatten()                   # for comparing it with true_labels
    return accuracy_score(labels_flat, preds_flat)

In [19]:

for epoch in tqdm(range(1, epochs+1)): # As the range function accounts for one value less than the original, we have epoch+1
    model.train()                      # calling the fucntion to train the model.
    
    loss_train_total=0                 # initialising the loss at starting to be zero. 
    
    progress_bar= tqdm(dataloader_train, 
                       desc='Epoch {:1d}'.format(epoch),
                       leave=False,
                       disable=False
                      )               # Progress bar helps us to track the number of epochs remaining out of total epochs.         
    for batch in progress_bar:
        
        model.zero_grad()             # Setting gradient to zero when strating the training
        
        batch= tuple(b.to(device) for b in batch)
        
        inputs={
            'input_ids'     : batch[0],
            'attention_mask': batch[1],        # passing the input prameters
            'labels'        : batch[2]
        }
        
        outputs= model(**inputs)
        
        loss= outputs[0]
        loss_train_total += loss.item()  #calculating total loss
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()               # calling our optimizer function defined above
        scheduler.step()               # calling our scheduler function defined above
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))}) # print training loss
        
    torch.save(model.state_dict(), f'/content/gdrive/My Drive/Colab Notebooks/Masters Project/VMDataset/Models/BERT_ft_epoch{epoch}.model') # saving all the model with epoch number as a name.
    
    tqdm.write('\nEpoch {}'.format(epoch))                          # tqdm.write will print this statement at output
    
    loss_train_avg= loss_train_total/len(dataloader_train) # average train loss
    tqdm.write(f'Training loss: {loss_train_avg}')         # print training loss on screen
    
    val_loss, predictions, true_vals = evaluate(dataloader_test)
    val_acc= scoring_func(predictions, true_vals) # calculating f1_score user defined function.
    tqdm.write(f'Validation loss: {val_loss}')    # printing validation loss and F1 score weighted.
    tqdm.write(f'Accuracy Score: {val_acc}')


Epoch 1
Training loss: 0.5247582759602124



Validation loss: 0.5507973333199819
Accuracy Score: 0.8245614035087719



Epoch 2
Training loss: 0.3194850672821348



Validation loss: 0.6162606957885954
Accuracy Score: 0.8491228070175438



Epoch 3
Training loss: 0.1205807601529321


...


Validation loss: 1.3724508417977228
Accuracy Score: 0.8175438596491228



Epoch 7
Training loss: 0.004269259663289793



Validation loss: 1.1063842409186893
Accuracy Score: 0.856140350877193



Epoch 8
Training loss: 9.29773375469042e-05



Validation loss: 1.1221046083503299
Accuracy Score: 0.8596491228070176



In [20]:
preds_flat= np.argmax(predictions, axis=1).flatten() # as prediction array is flattened to 0 or 1, possible labels
labels_flat=true_vals.flatten()   
(preds_flat==labels_flat).mean()

0.8596491228070176

### **Load and Predict on Best Model**

In [21]:
best_model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(df.label.unique()),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

best_model.load_state_dict(torch.load('/content/gdrive/My Drive/Colab Notebooks/Masters Project/VMDataset/Models/BERT_ft_epoch8.model'))

device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
best_model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [22]:
dataloader_test = DataLoader(
    dataset_test, 
    sampler=SequentialSampler(dataset_test), 
    batch_size=4
    )

In [23]:
import torch.nn.functional as F

def predict(dataloader_test):
  
    best_model.eval()
    all_logits = []
    
    for batch in dataloader_test:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {
            'input_ids':      batch[0],
            'attention_mask': batch[1],
            }

        with torch.no_grad():        
            outputs = best_model(**inputs)
            
        # since we have no loss, the only thing returned is logits
        logits = outputs[0]
        all_logits.append(logits)
    
    all_logits = torch.cat(all_logits, dim=0)
    preds_flat = np.argmax(all_logits.cpu().numpy(), axis=1).flatten()

    probs = F.softmax(all_logits, dim=1).cpu().numpy()

    # get highest prob dimension as prediction
    
    return preds_flat, probs


In [24]:
Y_pred, probs=predict(dataloader_test)
from sklearn import metrics

print('F1 Score:',metrics.f1_score(Y_test,Y_pred),
      'Precision:',metrics.precision_score(Y_test,Y_pred),
      'Recall:',metrics.recall_score(Y_test,Y_pred),
      'Accuracy:',metrics.accuracy_score(Y_test,Y_pred))

F1 Score: 0.8701298701298701 Precision: 0.881578947368421 Recall: 0.8589743589743589 Accuracy: 0.8596491228070176
